In [24]:
from graphdriver.commons import results, setup
from graphdriver.utils import paths
import pandas as pd
from collections import defaultdict

In [17]:
def ranking_standardize_df(df, column_name: str):
    df.loc["mean"] = df.mean()
    df = df.applymap("{0:.3f}".format)
    df[column_name] = df["mean"] + "$\pm$" + df["std"]
    df = df[[column_name]]
    df.index = "\textbf{" + df.index.str.upper() + "}"
    return df

In [23]:
dfs = []
for nt in list(reversed(setup.network_types()))[1:]:
    scores_dict = defaultdict()
    for cancer in setup.cancer_types():
        score, std = results.load_results(cancer, nt).score_test_pr_auc()
        scores_dict[cancer] = {"mean": score, "std": std}
        df = pd.DataFrame(scores_dict).T
        df = ranking_standardize_df(df, '_'.join(nt))
    dfs.append(df)

df = pd.concat(dfs, axis=1)

In [25]:
# add deepdriver
df_deep = paths.pd_load(path=paths.results_deepdriver() + "results_summary")
df_deep.loc["mean"] = df_deep.mean()
df_deep = df_deep.applymap("{0:.3f}".format)
df_deep["column_name"] = df_deep["deepdriver_mean"] + "$\pm$" + df_deep["deepdriver_std"]
df["DeepDriver"] = df_deep["column_name"].to_list()

In [32]:
# add emogi
emogi = ['--' for _ in range(14)]
emogi[1] = '0.59*'
emogi[-3] = '0.27*'
df["Emogi"] = emogi

In [33]:
path = 'results_main.tex'
df.to_latex(path, escape=False)

/tmp/ipykernel_17245/123936173.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(path, escape=False)
